In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import json
import os
import datetime
import nrrd
import bg_space as bg
from pprint import pprint
from matplotlib.colors import LogNorm, Normalize
import seaborn as sb
import pathlib
from bg_atlasapi import BrainGlobeAtlas
from lcm_registration import LCM_registration_functions as lrf
from lcm_registration import visualign_functions as vis
from znamutils import slurm_it
from PIL import Image
import os
import yaml

(Optional) Before using QuickNII, you can run the below code to make LCM section images conform to requirements for lcm registration pipeline (convert images to jpg and making sure they start with '_s0')

In [ ]:
input_folder = "/nemo/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6d/LCM/sections_same_orientation/"
output_folder = "/nemo/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6d/LCM/sections_QuickNII/"

lrf.convert_tif_to_jpg(input_folder, output_folder)

In [ ]:
# remove mouse name from section images filename.
mouse_name = "FIAA45.6D"
for filename in os.listdir(input_folder):
    if filename.startswith(f"{mouse_name}_") and filename.endswith(".TIF"):
        # Construct the new filename by removing "FIAA45_"
        new_filename = filename.replace(f"{mouse_name}_", "")

        # Rename the file
        os.rename(
            os.path.join(input_folder, filename),
            os.path.join(input_folder, new_filename),
        )
        print(f"Renamed {filename} to {new_filename}")

After completing registration using QuickNII and Visualign softwhare, you can now calculate z distance between sections

In [ ]:
parameters_path = (
    "/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6a/Sequencing"
)

1. Convert each registered section image to numpy file containing 2D allen ccf coordinates

In [ ]:
convert_job = lrf.convert_images(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

2. Now find find the pixels in the previous section with closest Euclidean distance, in order to calculate z-distance between the two sections

In [ ]:
lrf.get_euclidean_distance(
    parameters_path=parameters_path,
    use_slurm=True,
    job_dependency=convert_job,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)
# lcm_dir = '/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6a/LCM'
# lrf.get_euclidean_distance(lcm_dir, use_slurm=False, slurm_folder='/camp/home/turnerb/slurm_logs')

(Optional) check how the z distance between sections looks

In [ ]:

parameters =lrf.load_parameters(directory=parameters_path)
lcm_dir = pathlib.Path(parameters["lcm_directory"])
%matplotlib inline
section = 44
euclid_dist = np.load(f'{lcm_dir}/allenccf/z_calc/euclid_distance_S0{section}.npy')
z_dist = np.load(f'{lcm_dir}/allenccf/z_calc/z_add_S0{section}.npy')
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
sb.heatmap(euclid_dist, ax=axs[0, 0])
axs[0, 0].set_title('Nearest Euclidian distance')
sb.heatmap(z_dist, ax=axs[0, 1])
axs[0, 1].set_title('Z distance')
if os.path.isfile(f'{lcm_dir}/sections_same_orientation/S0{section}.TIF'):
    image_to_look= f'{lcm_dir}/sections_same_orientation/S0{section}.TIF'
elif os.path.isfile(f'{lcm_dir}/sections_same_orientation/S0{section}.tif'):
    image_to_look= f'{lcm_dir}/sections_same_orientation/S0{section}.tif'
img1 = plt.imread(image_to_look)
axs[1, 0].imshow(img1)
axs[1, 0].axis('off')
axs[1, 0].set_title('Section looked at')
if os.path.isfile(f'{lcm_dir}/sections_same_orientation/S0{section+1}.TIF'):
    image_to_look_2= f'{lcm_dir}/sections_same_orientation/S0{section+1}.TIF'
elif os.path.isfile(f'{lcm_dir}/sections_same_orientation/S0{section+1}.tif'):
    image_to_look_2= f'{lcm_dir}/sections_same_orientation/S0{section+1}.tif'
img2 = plt.imread(image_to_look_2)

axs[1, 1].imshow(img2)
axs[1, 1].axis('off')
axs[1, 1].set_title('Section before')
plt.tight_layout()

3. Now send job to create 3D atlas containing all LCM ROIs

In [ ]:
lrf.get_roi_vol(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

4. Calculate volumes and generate brain region annotations for each sample

In [ ]:
group_ROI_coordinates(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

In [ ]:
xa

In [ ]:
parameters = lrf.load_parameters(directory=parameters_path)

In [ ]:
for i in parameters["rois_to_combine"]:
    print(i)

In [ ]:
# now load ROI's to visualise
ROI_path = pathlib.Path(lcm_dir) / "rois"
pixcoord = []
for i, axis in enumerate([allencoord_roixa, allencoord_roiya, allencoord_roiza]):
    pixel = np.array(np.round(axis / 10), dtype=int)
    pixel[pixel < 0] = 0
    pixel[pixel >= empty_frame.shape[i]] = 0
    pixcoord.append(pixel)
new_coord = np.zeros(pixcoord[0].shape)
z_add = 0
for stack in range(int(np.round(z_to_add / 10))):
    for i in range(pixcoord[0].shape[0]):
        for j in range(pixcoord[0].shape[1]):
            if pixcoord[0][i, j] != 0:
                new_coord[i, j] = (pixcoord[0][i, j]) - z_add
    z_add = z_add + 1
    for k in range(pixcoord[0].shape[0]):
        for l in range(pixcoord[0].shape[1]):
            x = new_coord[k, l]
            y = pixcoord[1][k, l]
            z = pixcoord[2][k, l]
            if x != 0 and y != 0 and z != 0:
                empty_frame[int(x), int(y), int(z)] = int(tube)

In [ ]:
# get e.g. of ROI
for ROI_to_look in os.listdir(ROI_path):
    # region = ROI_path/'s015_TUBE6.png'
    region = ROI_path / ROI_to_look
    if ROI_to_look.startswith("s0") or ROI_to_look.startswith("S0"):
        slicename = region.stem[1:4]
        tube = region.stem[5 : len(region.stem)].split("TUBE", 1)[1]
        # if int(tube) in cortical_samples_table['Tube'].to_list():
        [xa, ya, za, one] = np.load(reg_dir / f"allen_ccf_converted_s{slicename}.npy")
        roi = plt.imread(ROI_path / f"{region}")
        allencoord_roiya = roi * ya
        allencoord_roiza = roi * za
        allencoord_roixa = roi * xa
        z_dist = (
            np.load(f"{lcm_dir}/allenccf/allen_ccf_coord/z_add_s{slicename}.npy") / 10
        )  # convert to 10um pixels

        add_z.loc[add_z["slice"] == f"s{slicename}", "amountz"].iloc[0]

In [ ]:
reg_dir = pathlib.Path(f"{lcm_dir}/allenccf/allen_ccf_coord/")
z_averages = []
for file in reg_dir.glob("z_add_s*.npy"):
    z_file = np.load(file)
    z_av = np.mean(z_file)
    z_averages.append(z_av)
average_z = np.mean(z_averages)
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

mcc = MouseConnectivityCache(resolution=10)
avg_temp, meta = mcc.get_template_volume()
empty_frame = np.zeros((avg_temp.shape))

In [ ]:
# get e.g. of ROI
for ROI_to_look in os.listdir(ROI_path):
    # region = ROI_path/'s015_TUBE6.png'
    region = ROI_path / ROI_to_look
    if ROI_to_look.startswith("s0") or ROI_to_look.startswith("S0"):
        slicename = region.stem[1:4]
        tube = region.stem[5 : len(region.stem)].split("TUBE", 1)[1]
        # if int(tube) in cortical_samples_table['Tube'].to_list():
        [xa, ya, za, one] = np.load(reg_dir / f"allen_ccf_converted_s{slicename}.npy")
        roi = plt.imread(ROI_path / f"{region}")
        allencoord_roiya = roi * ya
        allencoord_roiza = roi * za
        allencoord_roixa = roi * xa
        z_file = reg_dir / f"z_add_s{slicename}.npy"
        if z_file.is_file():
            z_dist = np.load(z_file)

            z_dist_pix = np.round(z_dist / 10)
            # convert the x, y, z coordinates to pixel

            pixcoord = []
            for i, axis in enumerate(
                [allencoord_roixa, allencoord_roiya, allencoord_roiza]
            ):
                pixel = np.array(np.round(axis / 10), dtype=int)
                pixel[pixel < 0] = 0
                pixel[pixel >= empty_frame.shape[i]] = 0
                pixcoord.append(pixel)
            for i in range(pixcoord[0].shape[0]):
                for j in range(pixcoord[0].shape[1]):
                    if pixcoord[0][i, j] != 0:
                        for z_add in range(int(z_dist_pix[i, j])):
                            x = pixcoord[0][i, j] - z_add
                            y = pixcoord[1][i, j]
                            z = pixcoord[2][i, j]
                            if x != 0 and y != 0 and z != 0:
                                empty_frame[int(x), int(y), int(z)] = int(tube)
        elif z_file.is_file() == False:
            average_z_pix = np.round(average_z / 10)
            for i in range(pixcoord[0].shape[0]):
                for j in range(pixcoord[0].shape[1]):
                    if pixcoord[0][i, j] != 0:
                        for z_add in range(int(average_z_pix)):
                            x = pixcoord[0][i, j] - z_add
                            y = pixcoord[1][i, j]
                            z = pixcoord[2][i, j]
                            if x != 0 and y != 0 and z != 0:
                                empty_frame[int(x), int(y), int(z)] = int(tube)
                # new_coord[i,j] = (pixcoord[0][i, j])-z_add

In [ ]:
np.save(f"{lcm_dir}/ROI_flatmap.npy", empty_frame)